In [1]:
import ast

filenames=[]
datasets=[]
with open("train.txt","r",encoding="utf-8") as f:
    lines=f.readlines()
    
for line in lines:
    name,data=line.split("\t")
    filenames.append(name)
    datasets.append(ast.literal_eval(data))

In [2]:
from datasets import load_dataset
traindata=load_dataset("csv",data_files="datasets\大赛1000训练用数据集.csv")
mark=traindata["train"]

markdata={}
value={}
name="001bfce2288c0cbe0a2811"
lastname=""
for item in mark:
    if item["图名"]!=name:
        markdata[lastname]=value
        value={}
        name=item["图名"]
    value[item["属性名"]]=item["正确值"]
    lastname=item["图名"]

markdata[mark[-1]["图名"]]=value

Using custom data configuration default-d3fba8b5dbc43d09
Reusing dataset csv (D:\hugging-face\datasets\csv\default-d3fba8b5dbc43d09\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
for key,value in markdata.items():
    if "入院日期" in value.keys():
        value["住院日期"]=value["入院日期"]
        value["入院时间"]=value["入院日期"]
    if "出院日期" in value.keys():
        value["出院时间"]=value["出院日期"]
    if "住院天数" in value.keys():
        value["住院日数"]=value["住院天数"]
        value["共住院"]=value["住院天数"]
    if "复核人" in value.keys():
        value["复核"]=value["复核人"]
    if "（小写）" in value.keys():
        value["￥"]=value["（小写）"]
    if "票据号码" in value.keys():
        value["No"]=value["票据号码"]
        value["NO"]=value["票据号码"]

In [4]:
label1=["入院日期","住院日期","入院时间"]
label2=["出院日期","出院时间"]
label3=["住院天数","住院日数","共住院"]
label4=["复核人","复核"]
label5=["（小写）","￥"]
label6=["票据号码","NO","No"]

In [5]:
def split_bbox(bbox, text):
    text=text.replace("\u3000","空")
    text=text.replace(" ","空")
    words = list(text)
    token_bboxes = []
    token=[]
    l1,r1,r2,l2 = bbox
    x1=l1[0]
    x2=r1[0]
    y1=r1[1]
    y2=r2[1]
    unit_w = (x2-x1) / len(text)
    for idx, word in enumerate(words):
        curr_w = len(word) * unit_w
        word_bbox = [[x1,y1],[x1+curr_w,y1],[x1+curr_w,y2],[x1,y2]]
        token_bboxes.append(word_bbox)
        token.append(word)
        x1 += curr_w
    
    return token_bboxes,token

In [6]:
def merge_bbox(boxlist,textlist):
    s=""
    for item in textlist:
        s+=str(item)
    s.replace("空"," ")
    if boxlist==[] and s=="":
        return -1,-1
    x1=boxlist[0][0][0]
    x2=boxlist[-1][1][0]
    y1=boxlist[0][0][1]
    y2=boxlist[0][2][1]
    box=[[x1,y1],[x2,y1],[x2,y2],[x1,y2]]
    return box,s

In [7]:
def compare(s1,s2):
    index=[]
    max=0
    data=""
    for l in s2:
        windows=len(list(l))
        for i in range(len(s1)-windows+1):
            num=0
            for j in range(windows):
                if s1[i+j]==l[j]:
                    num+=1
            score=num/windows
            if score>max:
                index=[]
                index.append(i)
                index.append(i+windows-1)
                max=score
                data=l
        
    if max>=0.66:
        return index,data
    else:
        return -1,-1

In [8]:
marked={}
for i in range(len(datasets)):
    name=filenames[i]
    for key,value in markdata.items():
        if (key in name) and (name.split(".")[0]==key.split(".")[0]):
            marked[name]=value

In [9]:
with open("train_data/fp/label_name.txt","r",encoding="utf-8") as f:
    label_list=f.read().splitlines()
key_name=[]
for item in label_list:
    first=item.split("_")[0]
    if first not in key_name:
        key_name.append(first)
del key_name[0]

key_eng=["GENDER","AGE","NAME","OCODE","OTYPE","IN","OUT","DAY","PC","PH","JY","OPEN","SK","FH","JS","XX","SKDW",
        "YBLX","HJ","ZC","JC","HY","ZL","SS","WSCL","XY","ZYYP","ZCYF","YBZL","CW","HL","GH","QT","YBTC","TC",
        "XJ","ZH","GRZF","QTZF","ZFY","ZFE"]
        
token2label={}
for a,b in zip(key_name,key_eng):
    token2label[a]=b


In [10]:
with open("train_data/fp/label_list.txt","w",encoding="utf-8") as f:
    f.write("OTHER"+"\n")
    for key,value in token2label.items():
        f.write(value+"_KEY\n")
        f.write(value+"_VALUE\n")


In [11]:
def get_key2(dct, value):
    return [k for (k,v) in dct.items() if v == value]

In [12]:
from tqdm.auto import tqdm
result={}
bar=tqdm(range(len(datasets)))

for i in range(len(datasets)):
    name=filenames[i]
    temp=[]
    result[name]=temp
    ziduan=list(marked[name].keys())
    data=list(marked[name].values())

    link={}
    for a,b in zip(ziduan,data):
        link[a]=-1
        link[b]=-1
    
    num=0
    for j in range(len(datasets[i])):
        num+=1
        dic={"transcription":"","label":"","points":"","id":"","linking":[]}
        index1=-1
        index2=-1
        index1,p=compare(list(datasets[i][j][1][0]),ziduan)
        index2,q=compare(list(datasets[i][j][1][0]),data)
        r=""
        if p in label1:
            r=label1[0]
        elif p in label2:
            r=label2[0]
        elif p in label3:
            r=label3[0]
        elif p in label4:
            r=label4[0]
        elif p in label5:
            r=label5[0]
        elif p in label6:
            r=label6[0]
        else:
            r=p

        if index1!=-1 and index2!=-1:
            boxx,tokenn=split_bbox(datasets[i][j][0], datasets[i][j][1][0])
            if "：" in datasets[i][j][1][0] or ":" in datasets[i][j][1][0]:
                if boxx[index1[1]+2:]==[] or tokenn[index1[1]+2:]==[]:
                    dic["transcription"]=datasets[i][j][1][0]
                    dic["label"]="other"
                    dic["points"]=datasets[i][j][0]
                    dic["id"]=num
                    result[name].append(dic)
                    continue
                qbox,qtext=merge_bbox(boxx[index1[0]:index1[1]+2],tokenn[index1[0]:index1[1]+2])
                abox,atext=merge_bbox(boxx[index2[0]:index2[1]+1],tokenn[index2[0]:index2[1]+1])
                if qbox==-1 or abox==-1:
                    dic["transcription"]=datasets[i][j][1][0]
                    dic["label"]="other"
                    dic["points"]=datasets[i][j][0]
                    dic["id"]=num
                    result[name].append(dic)
                    continue
                dic["transcription"]=qtext
                dic["label"]=token2label[str(r)].lower()+"_key"
                dic["points"]=qbox
                dic["id"]=num
                link[p]=num
                result[name].append(dic)
                num+=1
                dic={"transcription":"","label":"","points":"","id":"","linking":[]}
                dic["transcription"]=atext
                dic["label"]=token2label[str(r)].lower()+"_value"
                dic["points"]=abox
                dic["id"]=num
                link[q]=num
                result[name].append(dic)
            else:
                if boxx[index1[1]+1:]==[] or tokenn[index1[1]+1:]==[]:
                    dic["transcription"]=datasets[i][j][1][0]
                    dic["label"]="other"
                    dic["points"]=datasets[i][j][0]
                    dic["id"]=num
                    result[name].append(dic)
                qbox,qtext=merge_bbox(boxx[index1[0]:index1[1]+1],tokenn[index1[0]:index1[1]+1])
                abox,atext=merge_bbox(boxx[index2[0]:index2[1]+1],tokenn[index2[0]:index2[1]+1])
                dic["transcription"]=qtext
                dic["label"]=token2label[str(r)].lower()+"_key"
                dic["points"]=qbox
                dic["id"]=num
                link[p]=num
                result[name].append(dic)
                num+=1
                dic={"transcription":"","label":"","points":"","id":"","linking":[]}
                dic["transcription"]=atext
                dic["label"]=token2label[str(r)].lower()+"_value"
                dic["points"]=abox
                dic["id"]=num
                link[q]=num
                result[name].append(dic)
        elif index1!=-1:
            dic["transcription"]=datasets[i][j][1][0]
            dic["label"]=token2label[str(r)].lower()+"_key"
            dic["points"]=datasets[i][j][0]
            dic["id"]=num
            link[p]=num
            result[name].append(dic)
        elif index2!=-1:
            if "医院名称" in ziduan and q==marked[name]["医院名称"]:
                dic["transcription"]=datasets[i][j][1][0][index2[0]:index2[1]+1]
                dic["label"]="name"+"_value"
                dic["points"]=datasets[i][j][0]
                dic["id"]=num
                link[q]=num
                result[name].append(dic)
                continue
            
            r=""
            f=get_key2(marked[name],q)
            if f[0] in label1:
                r=label1[0]
            elif f[0] in label2:
                r=label2[0]
            elif f[0] in label3:
                r=label3[0]
            elif f[0] in label4:
                r=label4[0]
            elif f[0] in label5:
                r=label5[0]
            elif f[0] in label6:
                r=label6[0]
            else:
                r=f[0]
            dic["transcription"]=datasets[i][j][1][0][index2[0]:index2[1]+1]
            dic["label"]=token2label[str(r)].lower()+"_value"
            dic["points"]=datasets[i][j][0]
            dic["id"]=num
            link[q]=num
            result[name].append(dic)
        else:
            dic["transcription"]=datasets[i][j][1][0]
            dic["label"]="other"
            dic["points"]=datasets[i][j][0]
            dic["id"]=num
            result[name].append(dic)
    
    
    list1=list(link.keys())
    for item in list1:
        if item in ziduan:
            if link[item]!=-1 and link[marked[name][item]]!=-1:
                index=[link[item],link[marked[name][item]]]
                result[name][link[item]-1]["linking"].append(index)
                result[name][link[marked[name][item]]-1]["linking"].append(index)


    for v in range(len(result[name])):
        first=result[name][v]["label"].split("_")[0]
        if first in label1:
            last=result[name][v]["label"].split("_")[1]
            result[name][v]["label"]=token2label[label1[0]].lower()+"_"+last
        if first in label2:
            last=result[name][v]["label"].split("_")[1]
            result[name][v]["label"]=token2label[label2[0]].lower()+"_"+last
        if first in label3:
            last=result[name][v]["label"].split("_")[1]
            result[name][v]["label"]=token2label[label3[0]].lower()+"_"+last
        if first in label4:
            last=result[name][v]["label"].split("_")[1]
            result[name][v]["label"]=token2label[label4[0]].lower()+"_"+last
        if first in label5:
            last=result[name][v]["label"].split("_")[1]
            result[name][v]["label"]=token2label[label5[0]].lower()+"_"+last
        if first in label6:
            last=result[name][v]["label"].split("_")[1]
            result[name][v]["label"]=token2label[label6[0]].lower()+"_"+last

    
    
    bar.update(1)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
import json
with open("train_data/fp/train/train.txt","a",encoding="utf-8") as file:
    for key,value in result.items():
        file.write(key+"\t"+json.dumps(value, ensure_ascii=False) + "\n")

In [14]:
import json
with open("train_data/fp/train/train.txt","r",encoding="utf-8") as f1:
    dataset=f1.readlines()
    num=0
    for item in dataset:
        num+=1
        if num%5==0:
            f2=open("train_data/fp/val/val.txt","a",encoding="utf-8")
            f2.write(item)
            f2.close()

将训练集中抽取的测试集给放到图像集里

In [15]:
import shutil
with open("train_data/fp/val/val.txt","r",encoding="utf-8") as f1:
    data=f1.readlines()
    for item in data:
        filename=item.split("\t")[0]
        shutil.copy("datasets/train/"+filename,"train_data/fp/val/img/"+filename)

'import shutil\nwith open("train_data/fp/val/val.txt","r",encoding="utf-8") as f1:\n    data=f1.readlines()\n    for item in data:\n        filename=item.split("\t")[0]\n        shutil.copy("datasets/train/"+filename,"train_data/fp/val/img/"+filename)'